# Get Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import numpy as np
from sklearn import metrics

# Get Data

This data is from Kaggle for the use case of fraud detection in credit card transactions. Columns are pretty generic, but all values are type float. Can imagine other manipulations were made to this dataset such as converting categorical data to numeric for the purpose of feeding into the auto encoder.

In [2]:
# Credit Card Fraud Data
# https://www.kaggle.com/code/shivamb/semi-supervised-classification-using-autoencoders/data
df = pd.read_csv(r'C:\Users\Aaron\.kaggle\creditcard.csv\creditcard.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [4]:
# 0 = Non Fraud, 1 = Fraud 
pd.DataFrame(df.groupby('Class').size()).rename(columns={0:'count'})

,count
Class,
0,284315
1,492


In [5]:
# Table Stats
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


# Prep Data for Training

This is where the semi-supervised aspect takes place. The "non-fraud" rows are separated from the fraud rows and each are respectively put into their own datasets. We will train the model on the non-fraud rows.  

In [6]:
# Separate Data and drop class labels
nonFraud = df[df['Class'] == 0].drop('Class',axis=1)
fraud = df[df['Class'] == 1].drop('Class',axis=1)

# Train Test Split for nonFraud Auto Encoder training
x_train, x_test = train_test_split(nonFraud,test_size=.35, random_state = 5)

# Build & Fit Model

Creating and fitting the auto encoder model. Both encoding and decoding layers consists of 25 neurons and the hidden layer contains 3. Essentially the model will compress the input data down into this 3 neuron latent representation of the data and then rebuild the data using the learned latent representation. Then in an iterative process we will train the model to reduce the difference from the input data and the output data built from the hidden layer. Our loss is based on the mean squared error and we trained the model for 50 iterations/epochs.

In [8]:
# Build Auto Encoder

modelAE = models.Sequential([
    layers.InputLayer(input_shape= x_train.shape[1]),
    layers.Dense(25,activation='relu'), # encoding layer
    layers.Dense(3,activation='relu'), # latent hidden layer
    layers.Dense(25,activation='relu'), # decoding layer
    layers.Dense(x_train.shape[1])
])

modelAE.compile(loss='mean_squared_error',optimizer='adam')
modelAE.fit(x_train,x_train,verbose=1,epochs=50)

Epoch 1/50
5776/5776 [==============================] - 10s 2ms/step - loss: 3599354.5000
Epoch 2/50
5776/5776 [==============================] - 9s 2ms/step - loss: 350.3138
Epoch 3/50
5776/5776 [==============================] - 9s 2ms/step - loss: 423.5440
Epoch 4/50
5776/5776 [==============================] - 10s 2ms/step - loss: 339.6102
Epoch 5/50
5776/5776 [==============================] - 10s 2ms/step - loss: 359.6619
Epoch 6/50
5776/5776 [==============================] - 10s 2ms/step - loss: 318.4827
Epoch 7/50
5776/5776 [==============================] - 9s 2ms/step - loss: 399.5427
Epoch 8/50
5776/5776 [==============================] - 10s 2ms/step - loss: 273.5452
Epoch 9/50
5776/5776 [==============================] - 10s 2ms/step - loss: 251.4909
Epoch 10/50
5776/5776 [==============================] - 10s 2ms/step - loss: 265.1780
Epoch 11/50
5776/5776 [==============================] - 10s 2ms/step - loss: 324.2635
Epoch 12/50
5776/5776 [============================

# Evaluate

In [9]:
# How does the model perform on new non-fraud data
pred = modelAE.predict(x_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,x_test))

# How does the model perform on all non-fraud data
pred = modelAE.predict(nonFraud)
score2 = np.sqrt(metrics.mean_squared_error(pred,nonFraud))

# How does the model perform on fraud data
pred = modelAE.predict(fraud)
score3 = np.sqrt(metrics.mean_squared_error(pred,fraud))
print(f"Out of Sample Non-Fraud (RMSE): {score1}")
print(f"Insample Non-Fraud Score (RMSE): {score2}")
print(f"Fraud Score (RMSE): {score3}")

Out of Sample Non-Fraud (RMSE): 2.025225702585351
Insample Non-Fraud Score (RMSE): 2.0763803448145226
Fraud Score (RMSE): 5.405681401914101


We can see from the results that the errors associated with the insample and out of sample non-fraud data are about the same. This means, using the hidden latent representation determined by the training data, the non-fraud data fed through is about the same producing about the same errors. However, the fraud data produces a greater error meaning that our hidden representation could not predict the fraud data as accuractely eluding to the fact that there is something different about this underlying data in comparison to the data that was used for training.